In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

In [3]:
torch.__version__


'2.3.1'

In [7]:
#hypyer parameters 
batch_size = 64
block_size = 256
max_iters = 5000

eval_interval = 500
learning_rate = 3e-4
device ='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

n_embs = 384
n_head = 6 
n_layer = 6
dropout = 0.2

In [8]:
torch.manual_seed(1337)

In [9]:
with open('../input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [22]:
# crate vocabulary list (sorted)
char = sorted(list(set(text)))
vocab_size = len(char)
vocab_size

65

In [23]:
char

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [28]:
stoi = {char: i for i, char in enumerate(char)}
itos = {i: char for i, char in enumerate(char)}


In [38]:
b = encoding('fuck you!')
b

[44, 59, 41, 49, 1, 63, 53, 59, 2]

In [65]:
encoding = lambda word: [stoi[i] for i in word]
# use '' to concatenate str 
decoding = lambda number: ''.join(itos[i] for i in number)

In [ ]:
#training ans tesing split 
